In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2


2023-11-28 14:45:23.552014: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
interpreter = tf.lite.Interpreter(model_path="../models/3.tflite")
interpreter.allocate_tensors()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
def draw_key_points(frame, keypoints, conf_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    i = 0
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > conf_threshold:
            cv2.circle(frame,(int(kx), int(ky)), 1, (0, 255, 0), 5)
            cv2.circle(frame,(int(10), int(10)), 2, (0, 255, 0), 5)
        if i == 0:
            print(ky, ky/720, kx, kx/1280)
        i += 1
    return frame


In [5]:
cap = cv2.VideoCapture(0)
dynamic_variable = 0
while cap.isOpened():
    ret, frame = cap.read()

    img = frame.copy()
    img = cv2.resize(img, (192, 192))
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192)
    input_image = tf.cast(img, dtype=tf.float32)

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]["index"], np.array(input_image))
    interpreter.invoke()
    key_points_with_scores = interpreter.get_tensor(output_details[0]["index"])

    draw_key_points(frame, key_points_with_scores, conf_threshold=0.5)

    # Define the text to be displayed
    dynamic_variable += 1

    text = f"Variable Value: {dynamic_variable}"

    # Coordinates where the text will appear
    text_position = (50, 50)

    # Font settings
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_color = (255, 255, 255)  # White color
    line_type = 2

    # Put the text on the frame
    cv2.putText(frame, text, text_position, font, font_scale, font_color, line_type)

    cv2.imshow("Live Feed Test", frame)

    if cv2.waitKey(10) & 0xFF==ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


457.85698413848877 0.6359124779701233 690.8810424804688 0.5397508144378662
495.2392530441284 0.6878322958946228 685.4041290283203 0.5354719758033752
488.7790775299072 0.6788598299026489 673.4296417236328 0.5261169075965881
487.37836360931396 0.676914393901825 676.4559173583984 0.5284811854362488
481.37914180755615 0.6685821413993835 671.3185119628906 0.5244675874710083
484.94802474975586 0.6735389232635498 674.1508483886719 0.5266803503036499
482.7132511138916 0.6704350709915161 671.7776489257812 0.5248262882232666
482.31534004211426 0.6698824167251587 667.5614929199219 0.521532416343689
477.55040645599365 0.6632644534111023 669.1733551025391 0.5227916836738586
477.1497058868408 0.6627079248428345 672.099609375 0.5250778198242188
438.45242500305176 0.6089617013931274 665.95947265625 0.5202808380126953
424.9753761291504 0.5902435779571533 673.6632537841797 0.5262994170188904
404.0187406539917 0.5611371397972107 671.7170715332031 0.5247789621353149
379.8054313659668 0.5275075435638428 67

-1